# Test du modèle final

In [331]:
import streamlit as st
import joblib
import pandas as pd
import numpy as np

In [332]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os

In [333]:
load_dotenv()

True

In [334]:
user = os.getenv("PGUSER")
password = os.getenv("PGPASSWORD")
host = os.getenv("PGHOST")
port = os.getenv("PGPORT")
database = os.getenv("PGDATABASE")

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [335]:
def import_table_to_dataframe(table_name):
    query = f"SELECT * FROM {table_name}"
    df = pd.read_sql(query, engine)
    return df

data_clean = import_table_to_dataframe("immo")
data_clean.shape

(1648, 23)

In [336]:
@st.cache_resource  # cache la ressource dans Streamlit pour accélérer
def load_model():
    model = joblib.load('model.pkl')
    return model

model = load_model()

In [337]:
print(model)

Pipeline(steps=[('model',
                 TransformedTargetRegressor(func=<ufunc 'log1p'>,
                                            inverse_func=<ufunc 'expm1'>,
                                            regressor=GradientBoostingRegressor(learning_rate=0.05,
                                                                                min_samples_split=5,
                                                                                n_estimators=200,
                                                                                random_state=42,
                                                                                subsample=0.8)))])


On choisit une ligne au dans notre dataset

In [338]:
n=78

In [339]:
test_array = data_clean.loc[[n], ['nb_tot', 'ville_encoded', 'ville_encoded_m2', 'Ascenseur', 'surface_area']]

# Faire la prédiction
prediction = model.predict(test_array)

# Afficher la prédiction et la réalité correspondante
print(f"Prediction : {prediction[0]:.2f}, Réalité : {data_clean.loc[n, 'price']:.2f} difference: {prediction[0]-data_clean.loc[n, 'price']:.2f}")

Prediction : 1454613.17, Réalité : 1580000.00 difference: -125386.83


On obtient la prévision et la vraie valeure ainsi que la différence